# **[Tensorflow] CBOW을 이용한 word2vec**

### **1. Data loading**

In [ ]:
import numpy as np
import pandas as pd
from konlpy.tag import Komoran

In [ ]:
train_df = pd.read_csv('./input/ratings_train.txt', delimiter='\t')
pos_df = train_df[train_df['label'] == 1]['document']
neg_df = train_df[train_df['label'] == 0]['document']

### **2. Data preprocessing**
#### **(1) Text를 word의 seq로 변경**

In [ ]:
komoran = Komoran()

In [ ]:
pos_data = []
pos_label = []
for pos in pos_df:
    try:
        words = komoran.nouns(pos)
        # 단어가 3개 이상 등장한 review 에 대해서만 데이터 set을 생성한다.
        if len(words) > 3:
            pos_data.append(words)
            # 긍정을 의미하는 label 이다.
            pos_label.append(1)
    except:
        pass

In [ ]:
neg_data = []
neg_label = []
for neg in neg_df:
    try:
        words = komoran.nouns(neg)
        if len(words) > 3:
            neg_data.append(words)
            neg_label.append(0)
    except:
        pass

In [ ]:
texts = pos_data + neg_data
labels = pos_label + neg_label

print(len(texts))
print(len(labels))

In [ ]:
texts[:5]

#### **(2) Word의 seq를 number의 seq로 변경 (word2vec의 input : 정수)**

In [ ]:
# Indexing dictionary
words = []
for text in texts:
    words.extend(text)

In [ ]:
from collections import Counter
voca_size = 25000
corpus = {}

# 빈도수가 높은 단어 순서대로, indexing이 된다.
for word, freq in Counter(words).most_common(voca_size):
    corpus[word] = len(corpus)

# 결과 디버깅을 위하여, number seq에서, number에 대응하는 word를 찾기 위한 lookup dictionary
corpus_rev = dict(zip(corpus.values(), corpus.keys()))

In [ ]:
# Number seq를 생성한다. 즉 단어에 대응되는 num의 seq로 바꾸는 작업
num_seqs = []

for text in texts:
    num_seq = []
    for word in text:
        if word in corpus:
            idx = corpus[word]
        else:
            idx = 0  # 없는 단어는 0으로 한다. voca_size를 지정했기 때문이다.
        num_seq.append(idx)
    num_seqs.append(num_seq)

In [ ]:
num_seqs[0]

#### **(3) Input batch (target word, source context word)의 tuple를 생성**
 - CBOW는 source context word를 이용하여 target word를 예측하는 학습과정을 가진다.
 - Source context word left, target word, source context right 이렇게 주어지면, 학습 모델은 source context left -> target word, source context right -> target word가 된다.

In [ ]:
# Seqs : [ src_ctx_left, target, src_ctx_right ]
# Inputs : [ [src_ctx_left, src_ctx_right], [target] ]
# Batch : [ src_ctx_left (+) src_ctx_right ], label : [ target ] 

# Window size는 src_ctx_left ~ target까지의 크기이다.
def generate_batch_data(num_seqs, batch_size, window_size):
    batch_data = []
    label_data = []
    
    while len(batch_data) < batch_size:
        # 시작점을 random하게 선택한다.
        random_seq = np.random.choice(num_seqs)
        sources_and_target = []
        for idx in range(window_size, len(random_seq) - window_size):
            target = random_seq[idx]
            sources = random_seq[idx-window_size : idx] + random_seq[idx+1 : idx+window_size+1]
            sources_and_target.append((sources, target))
            
        # Target word, source context word를 분리한다.
        sources_list, target_list = [list(x) for x in zip(*sources_and_target)]
        # 데이터를 생성한다.
        batch_data.extend(sources_list)
        label_data.extend(target_list)
    
    # batch 크기만 생성
    batch_data = batch_data[:batch_size]
    label_data = label_data[:batch_size]
    
    # Placeholder에 feeding하기 위해서는 numpy array 형태로!
    batch_data = np.array(batch_data)
    # Nce_loss에서 사용하기 위해서 shape을 변경한다.
    label_data = np.transpose(np.array([label_data]))
    
    return (batch_data, label_data)

In [ ]:
# Validation set을 생성한다. 생성한 embedding matrix와의 거리를 측정할 수 있다.
# 이것은 단순히 similarity를 보여주기 위한 예이다.
valid_words = ['한국', '사랑', '스릴러', '장르']
valid_words = [corpus[elem] for elem in valid_words]

### **3. Modeling using tensorflow**

In [ ]:
import tensorflow as tf
import numpy as np

#### **(1) constants**

In [ ]:
batch_size = 128
embedding_size = 128
cbow_window = 1  # How many words to consider left and right
num_sampled = 64  # Number of negative examples to sample
learning_rate = 0.001
num_steps = 50000

In [ ]:
# Embedding 초기화
embeddings = tf.Variable(tf.random_uniform([voca_size, embedding_size], -1.0, 1.0))

In [ ]:
# Source context word
train_inputs = tf.placeholder(tf.int32, shape=[batch_size, 2*cbow_window])
# Target word
train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])

# 모델의 validation을 체크해본다.
valid_dataset = tf.constant(valid_words, dtype=tf.int32)

In [ ]:
# CBOW는 source context word의 embedding을 모두 더한 값을 사용한다.
embed = tf.zeros([batch_size, embedding_size])
for idx in range(2*cbow_window):
    embed += tf.nn.embedding_lookup(embeddings, train_inputs[:, idx])

In [ ]:
nce_weights = tf.Variable(tf.truncated_normal([voca_size, embedding_size],
                                              stddev = 1.0 / np.sqrt(embedding_size)))
nce_biases = tf.Variable(tf.zeros([voca_size]))
loss = tf.reduce_mean(tf.nn.nce_loss(weights=nce_weights,
                                     biases=nce_biases,
                                     labels=train_labels,
                                     inputs=embed,
                                     num_sampled=num_sampled,
                                     num_classes=voca_size))

In [ ]:
# Embedding 동작 과정을 확인하기 위하여 코사인 유사도를 이용
norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keepdims=True))
normalized_embeddings = embeddings / norm
valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_words)
similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)

In [ ]:
# optimizer 정의
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)

In [ ]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [ ]:
# train
for i in range(num_steps):
    batch_inputs, batch_labels = generate_batch_data(num_seqs, batch_size, cbow_window)
    feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}
    sess.run(optimizer, feed_dict=feed_dict)
    
    # loss
    if i % 100 == 0:
        print("=================================")
        loss_val = sess.run(loss, feed_dict=feed_dict)
        # Githup에 올릴때 주석, 결과를 보고 싶으면 주석을 해제하면 된다.
        print('Loss at step {}: {}'.format(i, loss_val))
        
        # Validation word와의 similarity를 구한다.
        sim = sess.run(similarity, feed_dict=feed_dict)
        for idx, num in enumerate(valid_words):
            valid_word = corpus_rev[num]
            top_k = 5
            nearest = (-sim[idx, :]).argsort()[1:top_k+1]
            log_str = 'Nearest to {}:'.format(valid_word)
            for k in range(top_k):
                close_word = corpus_rev[nearest[k]]
                log_str = '{} {},'.format(log_str, close_word)
            # Githup에 올릴때 주석, 결과를 보고 싶으면 주석을 해제하면 된다.
            print(log_str)

In [ ]:
sess.close()